# Example 0: Convert external data

In this example we will download external data that will be used to estimate relative moment tensors for seismic aftershocks related to the 2015-2017 Pamir earthquake sequence and convert them to the data formats required by *relMT*. For background information on the data set, please refer to [Bloch et al. (2023, GJI)](https://doi.org/10.1093/gji/ggac473).

> **Note**
>
> This tutorial relies on the command line tools `cd` `mkdir` and `wget`.
>
> They are part of most operating systems, but not all.
>
> `head` and `ls` are also used, but not required.

We will start by importing modules required for this tutorial.

In [1]:

from relmt import io, extra, mt, utils

## Download external data

We create a folder `ext/` to store the external data. For this example, we access the data publications

Bloch et al. (2022) Earthquake and Moment Tensor Catalogs of the 2015-2017 Pamir Earthquake Sequence. *GFZ Data Services*. https://doi.org/10.5880/fidgeo.2022.007

and

Bloch et al. (2026): Earthquake Phase Arrival Times of the 2015-2017 Pamir Earthquake Sequence. GFZ Data Services. https://doi.org/10.5880/fidgeo.2026.006

Let us create the directory, change into it and download the data.

In [2]:
# Make the directory, no warning if exists
! mkdir -p ext data

# -nc avoids re-downloading, -nv is less verbose, -P sets target directory
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_seismic_event_catalog.txt
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_moment_tensor_catalog_correct_norm_v2.0.txt
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2026.006-Kacenu/2026-006_Bloch-et-al_2015-2017_pamir_catalog_bulletin.xml


2026-02-02 10:17:47 URL:https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_seismic_event_catalog.txt [1885974/1885974] -> "ext/2022-007_Bloch-et-al_seismic_event_catalog.txt" [1]


2026-02-02 10:17:48 URL:https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_moment_tensor_catalog_correct_norm_v2.0.txt [5200/5200] -> "ext/2022-007_Bloch-et-al_moment_tensor_catalog_correct_norm_v2.0.txt" [1]


2026-02-02 10:17:49 URL:https://datapub.gfz.de/download/10.5880.FIDGEO.2026.006-Kacenu/2026-006_Bloch-et-al_2015-2017_pamir_catalog_bulletin.xml [53511277/53511277] -> "ext/2026-006_Bloch-et-al_2015-2017_pamir_catalog_bulletin.xml" [1]


## Create a *relMT* event file

We will now create a *relMT* compliant event file. Let us have a look at the provided catalog file.

In [3]:

! head ext/2022-007_Bloch-et-al_seismic_event_catalog.txt

# License: Creative Commons Attribution 4.0 International License (CC BY 4.0), https://creativecommons.org/licenses/by/4.0/
# Citation: Bloch, W.; Schurr, B.; Yuan, X.; Xu, Q.; Zhao, J.; Murodkulov, S.; Oimuhammadzoda, I. (2022): Earthquake and Moment Tensor Catalogs of the 2015--2017 Pamir Earthquake Sequence. GFZ Data Services. https://doi.org/10.5880/fidgeo.2022.007
#
#Year Month Day Hour Minute Second Timestamp(s)   Longitude(deg) Latitude(deg) Depth(km) P-picks S-picks revised? Method   RMS(s) Magnitude error type Sequence
 2015 08    03  14   37     08.00  1438612628.00  73.95270       38.48570      130.39      9         6   0        simulps  0.280  3.70      0.42  ML   Z       
 2015 08    04  02   10     29.00  1438654229.00  71.71370       37.19950      135.87      8         0   0        simulps  0.270  4.30      0.33  ML   Z       
 2015 08    04  22   19     21.00  1438726761.00  76.81780       39.48380       -3.00      6         0   0        simulps  0.330  2.37      0.33  

We find that the seismic event catalog contains the locations of the events in columns 7, 8 and 9 (0-based indexing), the origin time in column 6 (as a time stamp in seconds since 1970), and the magnitude in column 15. Some of the tools of the `extra` module will help us to:

* Convert the locations from geographic (degree) to UTM (meter) coordinates
* Provide a date string as an event name
* Save it in the `data/` subdirectory for use in the later examples

The Pamir highlands are located in [UTM zone 43 S](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system), in the Tajik Chinese border region.

In [4]:
# The external seismic event catalog file
catf = "ext/2022-007_Bloch-et-al_seismic_event_catalog.txt"

# Geoconverter for UTM zone 43S (Pamir region)
def geoconv(lat, lon, dep):
    return extra.geoconverter_latlon2utm(lat, lon, dep, 43, "S")


# Event name is the origin time in YYYYMMDDHHMMSS format
def nameconv(timetuple):
    yr, mo, da, hr, mn, sc = timetuple
    return f"{yr}{mo}{da}{hr}{mn}{sc[:2]}"


# Read the seismic event catalog into an event dictionary
evd = io.read_ext_event_table(
    catf,
    *(8, 7, 9, 6, 15),  # north, east, depth, magnitude, time
    (0, 1, 2, 3, 4, 5), # year, month, day, hour, minute, second
    geoconv,
    nameconverter=nameconv,
)

# And save it to file for later use
io.write_event_table(evd, "data/events.txt")

print(f"The seismic event catalog has {len(evd)} records.")
print("The first event is:")
print(evd[0])

The seismic event catalog has 11784 records.
The first event is:
Event(north=np.float64(4260226.228515374), east=np.float64(408658.4556633276), depth=np.float64(130389.99999999999), time=1438612628.0, mag=np.float64(3.7), name='20150803143708')


## Create a reference MT file

Next we will convert the moment tensor file. First, we inspect the file.

In [5]:
! head ext/2022-007_Bloch-et-al_moment_tensor_catalog_correct_norm_v2.0.txt

#Date      Time     Longitude(deg) Latitude(deg) CentroidDepth(km) Mw   mrr    mtt    mff    mrt    mrf    mtf   exp Strike(deg) Dip(deg)    Rake(deg)
2015/12/07 10:34:22 72.90430       38.28920      9                 4.4  0.897 -3.776  2.880  0.386  1.884 -3.049 22  26          81          24         
2015/12/07 15:23:57 73.22530       38.71950      4                 4.6 -1.816 -2.846  4.662  6.116 -0.731 -2.215 22  198         40          -16        
2015/12/27 23:05:28 72.69670       38.06920      6                 4.2 -1.633  0.083  1.550 -0.955 -0.228  0.725 22  181         40          -126       
2016/01/13 21:37:37 73.32220       38.74230      9                 4.8 -0.594 -0.782  1.376  0.489 -0.965  0.211 23  225         40          -22        
2016/03/18 16:11:00 72.61820       38.00300      4                 5.3 -0.025 -0.854  0.879  0.217 -0.339 -0.221 24  219         68          5          
2016/03/21 05:32:28 72.58100       38.00170      4                 4.1 -0.740 -0.285

The moment tensor catalog contains the components of the moment tensor. The format follows the Harvard convention used by the [Global CMT project](https://docs.generic-mapping-tools.org/latest/supplements/seis/meca.html) and documented e.g. in [GMT meca -Sm](https://docs.generic-mapping-tools.org/latest/supplements/seis/meca.html). The elements of the moment tensor in [dyne cm] units are described in columns 6 to 12 (0-based indexing).

For use in *relMT* we will next
* Convert the event time to a string that matches the event name of the catalog
* Convert from Harvard (R, $\theta$, $\phi$) to North-East-Down coordinates
* Convert from dyne cm to Newton meter (Nm)
* Write the moment tensor table

In [6]:

# The external moment tensor catalog file
mtf = "ext/2022-007_Bloch-et-al_moment_tensor_catalog_correct_norm_v2.0.txt"

# Event date times in the catalog
evdatetimes = [ev.name for ev in evd.values()]

# Function to convert date and time strings to event names
def nameconverter(datestr, timestr):
    # Remove delimiters from date and time strings and concatenate
    date = datestr.replace("/", "")
    time = timestr.replace(":", "")
    
    # Times do not match exactly, so look up closest event time
    datetime = f"{date}{time}"
    evdatetime = utils.approx_time_lookup([datetime], evdatetimes)

    # Look up in single-element dict
    return evdatetime[datetime]

# Function to convert moment tensors from Harvard format to north-east-down in Nm
def mtconverter(mrr, mtt, mff, mrt, mrf, mtf, exp):
    # Convert to north, east, down ...
    mnn, mee, mdd, mne, mnd, med = mt.rtf2ned(mrr, mtt, mff, mrt, mrf, mtf)

    #  ... and from dyne cm to Nm
    fac = 10**exp * 1e-7
    return (mnn * fac, mee * fac, mdd * fac, mne * fac, mnd * fac, med * fac)

# Read the moment tensor catalog into a moment tensor dictionary
mtd = io.read_ext_mt_table(
    mtf,
    (6, 7, 8, 9, 10, 11),  # mrr, mtt, mff, mrt, mrf, mtf
    (0, 1), # Date strings
    evd,
    mtconverter,  # This mtconverter converts from Harvad to north-east-down convention
    12,  # exponent
    nameconverter,  # The names are used to associate MTs to events
)

# And save it to disk
io.write_mt_table(mtd, "data/reference_mts.txt")

print(f"The MT catalog has {len(mtd)} records, namely:")
print(", ".join(map(str, mtd.keys())))

The MT catalog has 33 records, namely:
498, 515, 877, 972, 2050, 2190, 2695, 4181, 4189, 4273, 4370, 4414, 4472, 4569, 4664, 4839, 5037, 5508, 5883, 5888, 6101, 6102, 7440, 7508, 7640, 8919, 9617, 10715, 10847, 11447, 11481, 11551, 11685


## Conclusion
Let's have a look at the catalog and reference moment tensor file we just created. They now obey the *relMT* conventions, that is:
* units are in meter
* the axes point north, east and down, and
* a global event index (first column) is used to refer to the events

In [7]:
! head data/events.txt

#Number     Northing      Easting       Depth         Origintime Magnitude                 Name
# (int)      (meter)      (meter)     (meter)          (seconds)       (-)                (str)
      0  4260226.229   408658.456  130390.000  1438612628.000000    3.7000       20150803143708
      1  4122064.455   208320.271  135870.000  1438654229.000000    4.3000       20150804021029
      2  4372043.832   656334.005   -3000.000  1438726761.000000    2.3700       20150804221921
      3  4317525.747   520953.300   27690.000  1438744568.000000    2.4000       20150805031608
      4  4187827.608   249499.335   96070.000  1438844052.000000    3.0200       20150806065412
      5  4245191.012   388998.036  126440.000  1438846443.000000    3.0700       20150806073403
      6  4112039.663   219505.413   92410.000  1438868721.000000    3.3400       20150806134521
      7  4321404.819   518995.792   35030.000  1438931919.000000    2.8800       20150807071839


In [8]:
! head data/reference_mts.txt

#Number            nn            ee            dd            ne            nd            ed
# (int)          (Nm)          (Nm)          (Nm)          (Nm)           (Nm)         (Nm)
    498 -3.776000e+15  2.880000e+15  8.970000e+14  3.049000e+15  3.860000e+14 -1.884000e+15
    515 -2.846000e+15  4.662000e+15 -1.816000e+15  2.215000e+15  6.116000e+15  7.310000e+14
    877  8.300000e+13  1.550000e+15 -1.633000e+15 -7.250000e+14 -9.550000e+14  2.280000e+14
    972 -7.820000e+15  1.376000e+16 -5.940000e+15 -2.110000e+15  4.890000e+15  9.650000e+15
   2050 -8.540000e+16  8.790000e+16 -2.500000e+15  2.210000e+16  2.170000e+16  3.390000e+16
   2190 -2.850000e+14  1.025000e+15 -7.400000e+14 -5.040000e+14  3.330000e+14  7.900000e+14
   2695 -4.580000e+14 -1.631000e+15  2.088000e+15  3.822000e+15  3.700000e+14  2.683000e+15
   4181 -7.984000e+15  1.471000e+15  6.512000e+15  2.587000e+15 -2.013000e+15 -2.103000e+15
